In [ ]:
import onnxruntime as ort
import numpy as np
import cv2

# Load the ONNX model
session = ort.InferenceSession("C:/Users/okeiy/OneDrive - University of Salford/Documents/Sch Notes/Dissertation/Codes/Dataset/Yolo/bdd100k/runs/detect/train/weights/best.onnx")

# Get the name of the model's input
input_name = session.get_inputs()[0].name
print("Input Name:", input_name)

# Load the image
img_path = r"C:\Users\okeiy\OneDrive - University of Salford\Documents\Sch Notes\Dissertation\Codes\Dataset\Yolo\bdd100k\test\hudson-hintze-TCq5EVSIZbQ-unsplash.jpg"
orig_img = cv2.imread(img_path)  # Original image in BGR format

# Preprocess the image for the model
img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)  # Convert to RGB
img = cv2.resize(img, (640, 640))                  # Resize to 640x640
img = img.astype(np.float32) / 255.0               # Normalize to [0,1]
img = np.transpose(img, (2, 0, 1))                 # Change to (C, H, W)
img = np.expand_dims(img, axis=0)                  # Add batch dimension

# Run inference
outputs = session.run(None, {input_name: img})
print("Output Shapes:")
for output in outputs:
    print(np.shape(output))

# Resize the image to 640x640 pixels
resized_img = cv2.resize(img, (640, 640))

# To display the original image (or any image), convert it back to RGB if needed:
cv2.imshow("Test Image", orig_img)  # If you want to display the original image (BGR is fine with cv2.imshow)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
import cv2
import numpy as np
import onnxruntime as ort

# Load the ONNX model
session = ort.InferenceSession("C:/Users/okeiy/OneDrive - University of Salford/Documents/Sch Notes/Dissertation/Codes/Dataset/Yolo/bdd100k/runs/detect/train/weights/best.onnx")
input_name = session.get_inputs()[0].name
input_shape = session.get_inputs()[0].shape  # e.g., [1, 3, 640, 640]

# Define preprocessing function for a frame
def preprocess_frame(frame, target_size=(640, 640)):
    # Convert BGR to RGB
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    # Resize frame to target size
    frame = cv2.resize(frame, target_size)
    # Normalize the image to [0,1]
    frame = frame.astype(np.float32) / 255.0
    # Transpose to (C, H, W)
    frame = np.transpose(frame, (2, 0, 1))
    # Add batch dimension (B, C, H, W)
    frame = np.expand_dims(frame, axis=0)
    return frame

# Open video file or capture device
cap = cv2.VideoCapture(r"C:\Users\okeiy\Downloads\Video\123456.mp4")  # Use 0 for webcam

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Preprocess frame
    input_tensor = preprocess_frame(frame, target_size=(640, 640))

    # Run inference
    outputs = session.run(None, {input_name: input_tensor})

    # (Optional) Process outputs to get detection boxes, confidences, etc.
    # For example:
    # detections = postprocess(outputs)

    # (Optional) Visualize detections on the frame
    # Draw boxes, labels, etc. on 'frame' here

    # Display the frame
    cv2.imshow("Video Inference", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()